### A project by Villads Zimakoff

# Kuramoto-Sivashinsky in 1D
The kuramoto-Sivashinsky equation is a fourth order nonlinear PDE. It is used to model phenomena such as thermal instabilities in flame fronts and fluid dynamics. Mosty importantly for this project, the Kuramoto-Sivashinsky equation is known to exhibit chaotic properties highly sensitive to intial condtions within certain parameter domains. The focus of this project will be on deriving a numerical solution to the problem in 1D and 2D, aswell as investigate the behavior for given varrying domain sizes and intial conditions.  

Troughout this project i will use the notation:  
$\partial_{x  n} \rightarrow$ implies n times implicit differentiation of the given function with respect to x.  
$\ast \rightarrow$ will symbolize the convolution operator.  


 For the one dimensional function $u(x)$, the standard form for the Kuramoto-Sivashinsky equation is  
 
$$\begin{align*}
  \frac{du}{dt}   &= - \frac{1}{2} uu_{x} - u_{xx} - u_{xxxx}.  \\
\end{align*}$$

We then apply a quasi implicit discretization, relaxing the nonlinear term $uu_{x}$  

$$\begin{equation*}
  u_{t+\Delta t} = u_{t} - \Delta t(\frac{1}{2}u_{t} \partial_{x} u_{t} +\partial_{xx} u_{t+\Delta t} + \partial_{xxxx} u_{t+\Delta t}). \\ 
 \end{equation*}$$
 
Next we transform our equation from the spatial domain $u(x)$ into the frequency domain $\hat u(k)$ via the Fourier transformation. Note that any multiplicative term here transforms to become a convolution. 
  
  $$\begin{equation*}
\hat u_{t+\Delta t} = \hat u_{t} - \Delta t(\frac{1}{2} ( \hat u_{t} \ast ik_{x}\hat u_{t}) -k_{x}^2 \hat u_{t+\Delta t} + k_{x}^4\hat u_{t+\Delta t}). 
 \end{equation*}$$  
 
We then factor the implicit terms and isolate for the solution 

  $$\begin{equation*}
(1+\Delta t(k_{x}^4 - k_{x}^2 ))\hat u_{t+\Delta t} = \hat u_{t} - \Delta t\frac{1}{2}  (\hat u_{t} \ast ik_{x}\hat u_{t}) \\
\hat u_{t+\Delta t} = \frac{\hat u_{t} - \Delta t\frac{1}{2}  (\hat u_{t} \ast ik_{x}\hat u_{t})}{1+\Delta t(k_{x}^4 - k_{x}^2 )}. 
 \end{equation*}$$
 
But as the convolution operation is rather computationally expensive, we instead exploit the Fourier transform once again:

$$\begin{equation*}
\hat u_{t} \ast ik_{x}\hat u_{t} =  \mathcal{F} \{u_{t} \cdot \mathcal{F}^{-1} \{ ik_{x} \hat u_{t}\} \}.
\end{equation*}$$

To get the solution in the spatial domain we simple apply an inverse transformation. We now move to the implementation. Here we use a couple of "tricks" to further increase both speed and memory usage. Firstly the above term in the denominator is constant and can therefore be precomputed $\textit{once}$ and resued every loop. Secondly i use the pyfftw.interface to speed up numpy's Fourier transformations. 

In [26]:
pyfftw.interfaces.cache.enable()

def update(u, k_x, denominator, dt):
    u_trans = pyfftw.interfaces.numpy_fft.fft(u)
    
    u_x = np.real(pyfftw.interfaces.numpy_fft.ifft(1j * k_x * u_trans))
        
    nonlinear_term = dt * pyfftw.interfaces.numpy_fft.fft(u * u_x) / 2
    
    u_trans_next = ( u_trans - nonlinear_term  ) / denominator 
    
    u_new = np.real(pyfftw.interfaces.numpy_fft.ifft(u_trans_next))
    
    return u_new

def sim(N, L, u_i, t_max, dt):
    k_x = 2 * np.pi * fft.fftfreq(N, L / N)
    denominator = 1 + dt*(k_x**4 - k_x**2)
    
    u = u_i
    t, n = 0, 0
    u_sol, t_list = [], []
    while t < t_max:
        u = update(u, k_x, denominator, dt)
        if n % int(1/(10*dt)) == 0:
            u_sol.append(u)
            t_list.append(t)
        t += dt
        n += 1
        
    return np.array(u_sol), np.array(t_list)

# Kuramoto-Sivashinsky in 2D

 For the two dimensional function $u(x,y)$, the standard form for the Kuramoto-Sivashinsky equation is:  
 
$$\begin{align*}
  \frac{du}{dt}   &= - \frac{1}{2} |\nabla u|^{2} -\nabla^{2}u - \nabla^{4}u. \\
\end{align*}$$
To which we apply the quasi implicit discretization
$$\begin{equation*}
  u_{t+\Delta t} = u_{t} - \Delta t(\frac{1}{2} |\nabla u_{t}|^{2} +\nabla^{2}u_{t+\Delta t} + \nabla^{4}u_{t+\Delta t}) \\ 
  u_{t+\Delta t} = u_{t} - \Delta t(\frac{1}{2} ((\partial_{x} u_{t})^{2} + (\partial_{y} u_{t})^{2}) +(\partial_{xx} + \partial_{yy} )u_{t+\Delta t} + (\partial_{xxxx}  + \partial_{yyyy} + 2\partial_{xxyy})u_{t+\Delta t}). \\ 
 \end{equation*}$$
Note that since u is a scalar field, the absolute value of its gradient squared is
$|\nabla u_{t}|^{2} = (\sqrt{(\partial_{x} u_{t})^{2} + (\partial_{y} u_{t})^{2}})^{2} = (\partial_{x} u_{t})^{2} + (\partial_{y} u_{t})^{2}.$  

If we now fourier-transform
  
  $$\begin{equation*}
\hat u_{t+\Delta t} = \hat u_{t} - \Delta t(\frac{1}{2} ( (ik_{x} \hat u_{t} \ast ik_{x}\hat u_{t}) + (ik_{y} \hat u_{t} \ast ik_{y}\hat u_{t})) -(k_{x}^2 + k_{y}^2)\hat u_{t+\Delta t} + (k_{x}^4 + k_{y}^4 + 2k_{x}^{2}k_{y}^2)\hat u_{t+\Delta t}) \\
(1+\Delta t(k_{x}^4 + k_{y}^4 + 2k_{x}^{2}k_{y}^2 - k_{x}^2 - k_{y}^2))\hat u_{t+\Delta t} = \hat u_{t} - \Delta t\frac{1}{2}  ((ik_{x} \hat u_{t} \ast ik_{x}\hat u_{t}) + (ik_{y} \hat u_{t} \ast ik_{y}\hat u_{t})) \\
\hat u_{t+\Delta t} = \frac{\hat u_{t} - \Delta t\frac{1}{2}  ((ik_{x} \hat u_{t} \ast ik_{x}\hat u_{t}) + (ik_{y} \hat u_{t} \ast ik_{y}\hat u_{t}))}{1+\Delta t(k_{x}^4 + k_{y}^4 + 2k_{x}^{2}k_{y}^2 - k_{x}^2 - k_{y}^2)}. 
 \end{equation*}$$
 
Again we use the multiplicative property of the convolution in the spatial domain:

$$\begin{equation*}
ik_{x} \hat u_{t} \ast ik_{x}\hat u_{t} = \mathcal{F}\{ \mathcal{F}^{-1} \{ ik_{x} \hat u_{t}\} \cdot \mathcal{F}^{-1} \{ ik_{x} \hat u_{t}\} \}.
\end{equation*}$$


Taking the inverse Fourier transform we then obtain a solution and implementation analogue to before, but in two dimensions. It is however important, that since we are taking a two dimensional Fourier transformation, we handle our wave numbers and transformations carefully. This step crucial, as it is possible for energy to "leak" into the complex plane.  Further the domain size is asumed to be square so that $L_{x} = L_{y} = L$.  
  
Below is the implmentation.

In [17]:
import numpy as np

# pyfftw.interfaces.cache.enable()

def update(u, k_x, k_y, denominator, dt):
    # u_trans = pyfftw.interfaces.numpy_fft.rfft2(u)
    u_trans = np.fft.rfft2(u)

    # u_x = pyfftw.interfaces.numpy_fft.irfft2(1j * k_x * u_trans)
    # u_y = pyfftw.interfaces.numpy_fft.irfft2(1j * k_y * u_trans)
    u_x = np.fft.irfft2(1j * k_x * u_trans)
    u_y = np.fft.irfft2(1j * k_y * u_trans)
        
    # nonlinear_term = dt * pyfftw.interfaces.numpy_fft.rfft2(u_x * u_x + u_y * u_y) / 2
    nonlinear_term =  dt * np.fft.rfft2(u_x * u_x + u_y * u_y) / 2
    # nonlinear_term = np.zeros_like(nonlinear_term)

    u_trans_next = ( u_trans - nonlinear_term  ) / denominator 
    
    # u_new = pyfftw.interfaces.numpy_fft.irfft2(u_trans_next)
    u_new = np.fft.irfft2(u_trans_next)
    
    return u_new

def sim(N, L, u_i, t_max, dt):
    k_x, k_y = np.meshgrid(2 * np.pi * np.fft.rfftfreq(N, L/N), 2 * np.pi * np.fft.fftfreq(N, L/N))
    denominator = 1 + dt*(k_x**4 + k_y**4 + 2 * k_x**2 * k_y**2 - k_x**2 - k_y**2)
    
    print(k_x)
    print(k_y)


    u = u_i
    t, n = 0, 0
    u_sol = []
    while t < t_max:
        u = update(u, k_x, k_y, denominator, dt)
        # if n % int(1/(5*dt)) == 0:
        u_sol.append(u)
        t += dt
        n += 1
        
    return u_sol

In [18]:
t_max = 5.
dt = 0.005
L = 2 * np.pi
N = 16

# set initial condition
# u0 = sin(x) + sin(y) + sin(x + y)

x = np.linspace(0, L, N, endpoint=False)
y = np.linspace(0, L, N, endpoint=False)
xv, yv = np.meshgrid(x, y)
u0 = np.sin(xv) + np.sin(yv) + np.sin(xv + yv)

In [19]:
S=sim(N, L, u0, t_max, dt)

[[0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]
 [0. 1. 2. 3. 4. 5. 6. 7. 8.]]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.  2.]
 [ 3.  3.  3.  3.  3.  3.  3.  3.  3.]
 [ 4.  4.  4.  4.  4.  4.  4.  4.  4.]
 [ 5.  5.  5.  5.  5.  5.  5.  5.  5.]
 [ 6.  6.  6.  6.  6.  6.  6.  6.  6.]
 [ 7.  7.  7.  7.  7.  7.  7.  7.  7.]
 [-8. -8. -8. -8. -8. -8. -8. -8. -8.]
 [-7. -7. -7. -7. -7. -7. -7. -7. -7.]
 [-6. -6. -6. -6. -6. -6. -6. -6. -6.]
 [-5. -5. -5. -5. -5. -5. -5. -5. -5.]
 [-4. -4. -4. -4. -4. -4. -4. -4. -4.]
 [-3. -3. -3

In [21]:
S[-2]

array([[-3.06304387, -2.6943108 , -2.33736177, -2.05042556, -1.88359094,
        -1.86731166, -2.00461141, -2.27027866, -2.61740865, -2.98848234,
        -3.32679698, -3.58502792, -3.72989964, -3.7438356 , -3.6251867 ,
        -3.38822743],
       [-2.6943108 , -2.32557888, -1.96863299, -1.68170153, -1.51487251,
        -1.49859871, -1.63590288, -1.90157273, -2.24870316, -2.61977516,
        -2.95808636, -3.2163127 , -3.3611793 , -3.37511031, -3.25645729,
        -3.01949528],
       [-2.33736177, -1.96863299, -1.6116917 , -1.3247656 , -1.15794182,
        -1.14167222, -1.27897881, -1.54464892, -1.8917775 , -2.26284593,
        -2.60115248, -2.85937379, -3.00423569, -3.01816293, -2.89950754,
        -2.66254493],
       [-2.05042556, -1.68170153, -1.3247656 , -1.03784463, -0.87102485,
        -0.8547574 , -0.9920639 , -1.25773175, -1.60485635, -1.97591981,
        -2.31422119, -2.57243788, -2.71729628, -2.7312216 , -2.61256611,
        -2.37560526],
       [-1.88359094, -1.51487251, -1

In [25]:
np.fft.rfftfreq(N, L/N)

array([0.        , 0.15915494, 0.31830989, 0.47746483, 0.63661977,
       0.79577472, 0.95492966, 1.1140846 , 1.27323954])

In [27]:
2 * np.pi * np.fft.fftfreq(N, L/N)

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7., -8., -7., -6., -5., -4.,
       -3., -2., -1.])